#Q1

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/surya/NLP_Assignment/Programming Assignment - 3')

Mounted at /content/drive


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [ ]:
!pip install torch transformers datasets evaluate sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 13.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-many

In [ ]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=aa5d2fc87ef14dd9d56645769cd109023d3b3f2fc19bedcc446bc24368760333
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BartForConditionalGeneration, T5ForConditionalGeneration, BartTokenizer, T5Tokenizer
from transformers import BertTokenizer, EncoderDecoderModel, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import numpy as np
from tqdm import tqdm
import random


In [ ]:
def load_dataset(file_path_en, file_path_or):
    # Load data from files
    with open(file_path_en, 'r') as f_en, open(file_path_or, 'r') as f_or:
        en_texts = f_en.readlines()
        or_texts = f_or.readlines()

    # Ensure the lengths of the two files match
    assert len(en_texts) == len(or_texts), "Mismatch in the number of lines in the two files"

    # Strip newline characters and return the lists
    en_texts = [text.strip() for text in en_texts]
    or_texts = [text.strip() for text in or_texts]

    return en_texts, or_texts

# Load the dataset
en_texts, or_texts = load_dataset('/content/drive/MyDrive/surya/NLP_Assignment/Programming Assignment - 3/en-or/train.en', '/content/drive/MyDrive/surya/NLP_Assignment/Programming Assignment - 3/en-or/train.or')


In [ ]:
def split_dataset(en_texts, or_texts, train_size=100, val_size=10, test_size=10):
    # Combine English and Odia texts into pairs
    combined_data = list(zip(en_texts, or_texts))

    # Shuffle the data
    random.shuffle(combined_data)

    # Ensure that we have enough data for the specified sizes
    total_size = train_size + val_size + test_size
    assert len(combined_data) >= total_size, f"Insufficient data available for specified sizes: train_size={train_size}, val_size={val_size}, test_size={test_size}"

    # Split the data
    train_data = combined_data[:train_size]
    val_data = combined_data[train_size:train_size + val_size]
    test_data = combined_data[train_size + val_size:train_size + val_size + test_size]

    # Separate the pairs into English and Odia lists
    train_en, train_or = zip(*train_data)
    val_en, val_or = zip(*val_data)
    test_en, test_or = zip(*test_data)

    # Convert the tuples to lists and return
    return (list(train_en), list(train_or)), (list(val_en), list(val_or)), (list(test_en), list(test_or))

# Specify the sizes of train, validation, and test sets
train_size = 100  # Size of training set
val_size = 10     # Size of validation set
test_size = 10    # Size of test set

# Split the dataset
(train_en, train_or), (val_en, val_or), (test_en, test_or) = split_dataset(en_texts, or_texts, train_size=train_size, val_size=val_size, test_size=test_size)


In [ ]:
class TranslationDataset(Dataset):
    def __init__(self, src_texts, tgt_texts, tokenizer, max_length=512):
        self.src_texts = src_texts
        self.tgt_texts = tgt_texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.src_texts)

    def __getitem__(self, idx):
        src_text = self.src_texts[idx]
        tgt_text = self.tgt_texts[idx]

        # Tokenize source and target texts
        inputs = self.tokenizer(src_text, return_tensors='pt', max_length=self.max_length, truncation=True, padding='max_length')
        labels = self.tokenizer(tgt_text, return_tensors='pt', max_length=self.max_length, truncation=True, padding='max_length')

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': labels['input_ids'].squeeze(),
        }


In [ ]:
# Define tokenizers
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create dataset objects
train_dataset = TranslationDataset(train_en, train_or, bart_tokenizer)
val_dataset = TranslationDataset(val_en, val_or, bart_tokenizer)
test_dataset = TranslationDataset(test_en, test_or, bart_tokenizer)

# Create DataLoader objects
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

...

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Load pre-trained models
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')
bert_encoder = BertTokenizer.from_pretrained('bert-base-uncased')
bert_decoder = BertTokenizer.from_pretrained('bert-base-uncased')

# Define encoder-decoder model using BERT
bert_model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-uncased', 'bert-base-uncased')

# Send models to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bart_model.to(device)
t5_model.to(device)
bert_model.to(device)


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.layer.1.crossattention.self.key.bias', 'bert.e

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
def train_model(model, tokenizer, train_loader, val_loader, optimizer, device, num_epochs=10):
    model.train()
    best_loss = float('inf')

    for epoch in range(num_epochs):
        total_loss = 0
        for batch in tqdm(train_loader):
            inputs = batch['input_ids'].to(device)
            masks = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Reset gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(input_ids=inputs, attention_mask=masks, labels=labels)
            loss = outputs.loss

            # Backward pass
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Calculate average loss
        avg_loss = total_loss / len(train_loader)

        # Validate the model
        val_loss = validate_model(model, val_loader, device)

        # Print training and validation loss
        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_loss:.4f}, Validation Loss: {val_loss:.4f}')

        # Save the model if validation loss is the best
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')

def validate_model(model, val_loader, device):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            inputs = batch['input_ids'].to(device)
            masks = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids=inputs, attention_mask=masks, labels=labels)
            loss = outputs.loss

            total_loss += loss.item()

    # Calculate average validation loss
    avg_loss = total_loss / len(val_loader)

    return avg_loss


In [ ]:
# Define optimizers
bart_optimizer = torch.optim.Adam(bart_model.parameters(), lr=3e-5)
t5_optimizer = torch.optim.Adam(t5_model.parameters(), lr=3e-5)
bert_optimizer = torch.optim.Adam(bert_model.parameters(), lr=3e-5)

# Train BART model
train_model(bart_model, bart_tokenizer, train_loader, val_loader, bart_optimizer, device, num_epochs=3)

# Train T5 model
train_model(t5_model, t5_tokenizer, train_loader, val_loader, t5_optimizer, device, num_epochs=3)

# Train BERT model
train_model(bert_model, bert_tokenizer, train_loader, val_loader, bert_optimizer, device, num_epochs=3)


100%|██████████| 13/13 [00:14<00:00,  1.15s/it]


Epoch 1/3, Training Loss: 10.0863, Validation Loss: 8.0991


100%|██████████| 13/13 [00:13<00:00,  1.05s/it]


Epoch 2/3, Training Loss: 6.6975, Validation Loss: 4.3798


100%|██████████| 13/13 [00:13<00:00,  1.05s/it]


Epoch 3/3, Training Loss: 3.7041, Validation Loss: 2.9833


  0%|          | 0/13 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


#Q2

In [ ]:
!pip install optimum


In [ ]:
!pip install optimum auto-gptq


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import classification_report
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

# Load the GPT-3 model and tokenizer (or another LLM of your choice)
model_name = "togethercomputer/LLaMA-2-7B-32K"  # or choose another model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Function for zero-shot inference
def zero_shot_inference(model, tokenizer, test_data):
    predictions = []
    references = []

    # Loop through test data
    for en_text, or_text in test_data:
        # Encode the English text
        inputs = tokenizer(en_text, return_tensors="pt")

        # Perform inference using the model
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_length=50  # Adjust max_length as needed
            )

        # Decode the output to get the predicted Odia text
        predicted_or_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Append the predicted and reference texts to the lists
        predictions.append(predicted_or_text)
        references.append(or_text)

    return predictions, references

# Test the model using zero-shot inference
test_data = list(zip(test_en, test_or))
predictions, references = zero_shot_inference(model, tokenizer, test_data)

# Calculate performance metrics
# ROUGE score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = [scorer.score(ref, pred) for ref, pred in zip(references, predictions)]

# BLEU score
bleu_scores = [sentence_bleu([ref.split()], pred.split()) for ref, pred in zip(references, predictions)]

# Print the results
print("ROUGE Scores:", rouge_scores)
print("BLEU Scores:", bleu_scores)

# Classification report (if you have labeled test data for classification tasks)
# classification_report(references, predictions)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/942 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]